In [65]:
import pytensor 
import pymc as pm
import arviz as az
import pyhf
import numpy as np
import matplotlib.pyplot as plt
import json
import time

from pytensor import tensor as pt
from pytensor.graph.basic import Apply

# Prior-Predictive Checks for ttbar

We perform some prior-predictive checks for [this measurement](https://www.hepdata.net/record/ins1869695).

### Set-Up
 ... pyhf model set-up.

In [101]:
### Open the .json file
with open("ttbar_ljets_xsec_inclusive_pruned.json") as serialized:
    spec = json.load(serialized)

### Create pyhf model from it
workspace = pyhf.Workspace(spec)

model = workspace.model()

### Some model specs
nPars = len(model.config.suggested_init())
obs = workspace.data(model, include_auxdata=False)

... print the specifications.

In [ ]:
print(f"channels: {model.config.channels}")
print(".........")
print(f"nbins: {model.config.channel_nbins}")
print(".........")
print(f"samples: {model.config.samples}")
print(".........")
print(f"modifiers: {model.config.modifiers}")
print(".........")
print(f"parameters: {model.config.parameters}")
print(".........")
print(f"nauxdata: {model.config.nauxdata}")
print(".........")
print(f"auxdata: {model.config.auxdata}")

### Class

... Op creation.

In [77]:
### Class that creates the model Op
class Op(pt.Op):
    itypes = [pt.dvector]  # Expects a vector of parameter values
    otypes = [pt.dvector]  # Outputs a vector of values (the model.expected_actualdata)

    def __init__(self, name, func):
        ## Add inputs as class attributes
        self.func = func
        self.name = name

    def perform(self, node, inputs, outputs):
        ## Method that is used when calling the Op
        (theta,) = inputs  # Contains my variables

        ## Calling input function (in our case the model.expected_actualdata)
        result = self.func(theta)

        ## Output values of model.expected_actualdata
        outputs[0][0] = np.asarray(result, dtype=node.outputs[0].dtype)

## No Observations

In [75]:
### Applying the Op with arguments (function, name)
mainOp = Op("mainOp", model.expected_actualdata)

start_time = time.time()
### Opening the pyMC model space
with pm.Model() as basic_model:
    ## TensorVar input parameters [sigStr, gamma_i, ... ,gamma_nBins]
    pars = [pm.HalfNormal("sigStr", sigma=2)]
    pars.extend(pm.Gamma(f"gamma{idx}", alpha=20, beta=20) for idx in range(132))
    pars = pt.as_tensor_variable(pars)

    # main = pm.Poisson("main", mu=mainDet, observed=obs)
    # mainPoiss = pm.Poisson("mainPoiss", mu=mainOp(pars), observed=obs)
    mainPoiss = pm.Poisson("mainPoiss", mu=mainOp(pars))

    ## Sampling ...
    post_data = pm.sample(5)
    # prior_data = pm.sample_prior_predictive()
    # post_pred = pm.sample_posterior_predictive(post_data)

print("...............................................")
print(f"That took {(time.time() - start_time)/60} minutes ...")

Only 5 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CompoundStep
>>Slice: [sigStr]
>>Slice: [gamma0]
>>Slice: [gamma1]
>>Slice: [gamma2]
>>Slice: [gamma3]
>>Slice: [gamma4]
>>Slice: [gamma5]
>>Slice: [gamma6]
>>Slice: [gamma7]
>>Slice: [gamma8]
>>Slice: [gamma9]
>>Slice: [gamma10]
>>Slice: [gamma11]
>>Slice: [gamma12]
>>Slice: [gamma13]
>>Slice: [gamma14]
>>Slice: [gamma15]
>>Slice: [gamma16]
>>Slice: [gamma17]
>>Slice: [gamma18]
>>Slice: [gamma19]
>>Slice: [gamma20]
>>Slice: [gamma21]
>>Slice: [gamma22]
>>Slice: [gamma23]
>>Slice: [gamma24]
>>Slice: [gamma25]
>>Slice: [gamma26]
>>Slice: [gamma27]
>>Slice: [gamma28]
>>Slice: [gamma29]
>>Slice: [gamma30]
>>Slice: [gamma31]
>>Slice: [gamma32]
>>Slice: [gamma33]
>>Slice: [gamma34]
>>Slice: [gamma35]
>>Slice: [gamma36]
>>Slice: [gamma37]
>>Slice: [gamma38]
>>Slice: [gamma39]
>>Slice: [gamma40]
>>Slice: [gamma41]
>>Slice: [gamma42]
>>Slice: [gamma43]
>>Slice: [gamma44]
>>Slice: [gamma45]
>>Slice: [gamma46]
>>Sl

Sampling 4 chains for 1_000 tune and 5 draw iterations (4_000 + 20 draws total) took 586 seconds.


...............................................
That took 57.5561505039533 minutes ...


/var/folders/p9/sw_pjm3j027cxfs57s_91bfw0000gn/T/ipykernel_15489/3483040292.py:16: UserWarning: The number of samples is too small to check convergence reliably.
  post_data = pm.sample(5)


In [76]:
az.summary(post_data)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mainPoiss[0],1369564.250,11631.102,1356895.000,1382944.000,2649.879,1902.869,19.0,16.0,3.17
mainPoiss[1],817275.300,1993.711,815011.000,820110.000,454.221,326.175,19.0,19.0,4.67
mainPoiss[2],699002.150,4748.864,690675.000,702677.000,1081.919,776.923,19.0,16.0,5.29
mainPoiss[3],586394.400,4490.517,580533.000,592241.000,1023.061,734.657,19.0,19.0,3.65
mainPoiss[4],498942.050,6019.421,492900.000,507294.000,1371.386,984.788,19.0,19.0,4.58
...,...,...,...,...,...,...,...,...,...
gamma127,0.967,0.063,0.902,1.054,0.014,0.010,19.0,19.0,4.30
gamma128,0.959,0.039,0.915,1.019,0.009,0.006,19.0,19.0,3.23
gamma129,1.045,0.056,0.965,1.123,0.013,0.009,19.0,19.0,3.07
gamma130,1.015,0.037,0.971,1.077,0.009,0.006,19.0,19.0,2.88


## Include some observations

In [99]:
### Applying the Op with arguments (function, name)
mainOp = Op("mainOp", model.expected_actualdata)

### Opening the pyMC model space
start_time = time.time()
with pm.Model() as basic_model:
    ## TensorVar input parameters [sigStr, gamma_i, ... ,gamma_nBins]
    pars = [pm.HalfNormal("sigStr", sigma=2)]
    pars.extend(pm.Gamma(f"gamma{idx}", alpha=20, beta=20) for idx in range(132))
    pars = pt.as_tensor_variable(pars)

    mainPoiss = pm.Poisson("mainPoiss", mu=mainOp(pars), observed=obs)

    ## Sampling ...
    post_data = pm.sample(5)
    # prior_data = pm.sample_prior_predictive()
    # post_pred = pm.sample_posterior_predictive(post_data)

print("...............................................")
print(f"That took {(time.time() - start_time)/60} minutes ...")

Only 5 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [sigStr]
>Slice: [gamma0]
>Slice: [gamma1]
>Slice: [gamma2]
>Slice: [gamma3]
>Slice: [gamma4]
>Slice: [gamma5]
>Slice: [gamma6]
>Slice: [gamma7]
>Slice: [gamma8]
>Slice: [gamma9]
>Slice: [gamma10]
>Slice: [gamma11]
>Slice: [gamma12]
>Slice: [gamma13]
>Slice: [gamma14]
>Slice: [gamma15]
>Slice: [gamma16]
>Slice: [gamma17]
>Slice: [gamma18]
>Slice: [gamma19]
>Slice: [gamma20]
>Slice: [gamma21]
>Slice: [gamma22]
>Slice: [gamma23]
>Slice: [gamma24]
>Slice: [gamma25]
>Slice: [gamma26]
>Slice: [gamma27]
>Slice: [gamma28]
>Slice: [gamma29]
>Slice: [gamma30]
>Slice: [gamma31]
>Slice: [gamma32]
>Slice: [gamma33]
>Slice: [gamma34]
>Slice: [gamma35]
>Slice: [gamma36]
>Slice: [gamma37]
>Slice: [gamma38]
>Slice: [gamma39]
>Slice: [gamma40]
>Slice: [gamma41]
>Slice: [gamma42]
>Slice: [gamma43]
>Slice: [gamma44]
>Slice: [gamma45]
>Slice: [gamma46]
>Slice: [gamma47]
>Slice: [gamma48]
>Slice: [gamma49]
>Slice: [gam

/Users/malinhorstmann/anaconda3/envs/pymc_env/lib/python3.11/site-packages/pyhf/interpolators/code4p.py:94: RuntimeWarning: invalid value encountered in add
  deltas = tmp3_times_A + alphas_times_S
/Users/malinhorstmann/anaconda3/envs/pymc_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/malinhorstmann/anaconda3/envs/pymc_env/lib/python3.11/site-packages/pyhf/tensor/numpy_backend.py:281: RuntimeWarning: overflow encountered in power
  return np.power(tensor_in_1, tensor_in_2)
/Users/malinhorstmann/anaconda3/envs/pymc_env/lib/python3.11/site-packages/pyhf/interpolators/code4p.py:87: RuntimeWarning: overflow encountered in multiply
  tmp2 = asquare * tmp1 + 15.0
/Users/malinhorstmann/anaconda3/envs/pymc_env/lib/python3.11/site-packages/pyhf/interpolators/code4p.py:88: RuntimeWarning: overflow encountered in multiply
  tmp3 = asquare * tmp2
/Users/malinhorstm

ValueError: Not enough samples to build a trace.